In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

/opt/homebrew/lib/python3.11/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/np/m1q4q1w54q59sj70tvylvg5m0000gn/T/ipykernel_3980/1428030430.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. How

## Aggregation

In [2]:
public_transit = ['bk_bus', 'bus_company', 'bx_bus', 'LIRR', 'metro_north',
                    'mn_bus', 'nyc_subway', 'qn_bus', 'si_bus']

first = True
for transit in public_transit:

    stop = pd.read_csv("../data/landing/public_transit_data/"+transit+"/stops.txt")
    stop['geometry'] = stop.apply(lambda row: Point(row['stop_lon'], row['stop_lat']), axis=1)
    # stops = stops.drop(columns=['stop_name', 'stop_desc', 'zone_id', 'stop_url', 'location_type', 'parent_station'])
    stop = stop[['stop_id', 'geometry']]
    print(transit)
    print(stop.shape)
    # stop = stop.drop_duplicates(subset=['geometry'])
    if first:
        stops = stop
        first = False
    else:
        stops = pd.concat([stops, stop])
    # print(stop.shape)
print(stops.shape)
stops = stops.drop_duplicates(subset=['geometry'])
print(stops.shape)

bk_bus
(4619, 2)
bus_company
(3540, 2)
bx_bus
(1885, 2)
LIRR
(127, 2)
metro_north
(114, 2)
mn_bus
(1810, 2)
nyc_subway
(1497, 2)
qn_bus
(2094, 2)
si_bus
(1952, 2)
(17638, 2)
(16455, 2)


Note: We removed 1183 instances as they were duplicates.

In [3]:
bus_stops = gpd.GeoDataFrame(stops, geometry='geometry')

sf = gpd.read_file("../data/landing/taxi_zones/taxi_zones.shp")
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [4]:
# Join the public transport data to zone dataset using sjoin
result = gpd.sjoin(sf, bus_stops, how='left', op='contains')
grouped = result.groupby('LocationID')['stop_id'].count().reset_index()
grouped.rename(columns={'stop_id': 'num_stops'}, inplace=True)
final = sf.drop('OBJECTID', axis=1).merge(grouped, left_on='LocationID', right_on='LocationID')

/Users/idere_/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/np/m1q4q1w54q59sj70tvylvg5m0000gn/T/ipykernel_3980/1245944608.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs + ...
Right CRS: None

  result = gpd.sjoin(sf, bus_stops, how='left', op='contains')


In [5]:
# Output the cleaned public transport dataset
final.to_csv("../data/raw/public_transport.csv", index=False)